# Load Data

In [1]:
!wget -qO- https://drive.google.com/uc?id=1N-IQdlLay8VOJ4UI1JEAsEgHiiDvHJvD > Death.csv

In [2]:
!wget -qO- https://drive.google.com/uc?id=1s94ITS37-maKnsQaMYyfjyLUohvjBzm4 > Population_GDP.csv

In [3]:
!wget -qO- https://drive.google.com/uc?id=1x6X5neilmbM3JPCR3aeEjRdvcCRYIi_L > GDP_Population_Rate.csv

In [4]:
!wget -qO- https://drive.google.com/uc?id=1rHPa8nDXHj29wEfHl63wXTghW7rKNUrS > CO2_emission.csv

In [5]:
!wget -qO- https://drive.google.com/uc?id=1ijBxPTUmwRRveFtxLQxTbMYb5siETWOs > Enviroment_Risk.csv

In [6]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

In [7]:
death_pm_df = pd.read_csv('Death.csv')
death_pm_df.head()

,Entity,Code,Year,"Total population (Gapminder, HYDE & UN)",Continent,"PM2.5 air pollution, mean annual exposure (micrograms per cubic meter)",Deaths - Ambient particulate matter pollution - Sex: Both - Age: Age-standardized (Rate)
0,Abkhazia,OWID_ABK,2015,NaN,Asia,NaN,NaN
1,Afghanistan,AFG,1800,3280000.0,NaN,NaN,NaN
2,Afghanistan,AFG,1801,3280000.0,NaN,NaN,NaN
3,Afghanistan,AFG,1802,3280000.0,NaN,NaN,NaN
4,Afghanistan,AFG,1803,3280000.0,NaN,NaN,NaN


In [8]:
gdp_death_rate_df = pd.read_csv('Population_GDP.csv')
gdp_death_rate_df.head()

,Entity,Code,Year,"Total population (Gapminder, HYDE & UN)",Continent,"GDP per capita, PPP (constant 2011 international $)",Death rate – Outdoor air pollution (age-standardized) (IHME)
0,Abkhazia,OWID_ABK,2015,NaN,Asia,NaN,NaN
1,Afghanistan,AFG,1800,3280000.0,NaN,NaN,NaN
2,Afghanistan,AFG,1801,3280000.0,NaN,NaN,NaN
3,Afghanistan,AFG,1802,3280000.0,NaN,NaN,NaN
4,Afghanistan,AFG,1803,3280000.0,NaN,NaN,NaN


# Khám phá dữ liệu

In [9]:
death_pm_df['Year'].unique()

array([  2015,   1800,   1801,   1802,   1803,   1804,   1805,   1806,
         1807,   1808,   1809,   1810,   1811,   1812,   1813,   1814,
         1815,   1816,   1817,   1818,   1819,   1820,   1821,   1822,
         1823,   1824,   1825,   1826,   1827,   1828,   1829,   1830,
         1831,   1832,   1833,   1834,   1835,   1836,   1837,   1838,
         1839,   1840,   1841,   1842,   1843,   1844,   1845,   1846,
         1847,   1848,   1849,   1850,   1851,   1852,   1853,   1854,
         1855,   1856,   1857,   1858,   1859,   1860,   1861,   1862,
         1863,   1864,   1865,   1866,   1867,   1868,   1869,   1870,
         1871,   1872,   1873,   1874,   1875,   1876,   1877,   1878,
         1879,   1880,   1881,   1882,   1883,   1884,   1885,   1886,
         1887,   1888,   1889,   1890,   1891,   1892,   1893,   1894,
         1895,   1896,   1897,   1898,   1899,   1900,   1901,   1902,
         1903,   1904,   1905,   1906,   1907,   1908,   1909,   1910,
      

**Nhận xét:** Nhận thấy dữ liệu được thu thập từ năm 1700 nên ta chỉ lấy lấy dữ liệu từ năm 1990 đến nay.

## Lấy dữ liệu từ năm 1990 đến nay

In [10]:
def get_year(year):
  choice_year = np.arange(1990,2020)
  return year if year in choice_year else np.nan

In [11]:
death_pm_df['Year'] = death_pm_df['Year'].apply(get_year)
death_pm_df = death_pm_df[~death_pm_df['Year'].isna()].reset_index(drop=True)
death_pm_df.head()

,Entity,Code,Year,"Total population (Gapminder, HYDE & UN)",Continent,"PM2.5 air pollution, mean annual exposure (micrograms per cubic meter)",Deaths - Ambient particulate matter pollution - Sex: Both - Age: Age-standardized (Rate)
0,Abkhazia,OWID_ABK,2015.0,NaN,Asia,NaN,NaN
1,Afghanistan,AFG,1990.0,12412000.0,NaN,60.93669,46.446589
2,Afghanistan,AFG,1991.0,13299000.0,NaN,NaN,46.033841
3,Afghanistan,AFG,1992.0,14486000.0,NaN,NaN,44.243766
4,Afghanistan,AFG,1993.0,15817000.0,NaN,NaN,44.440148


## Đổi tên thuộc tính

In [12]:
death_pm_df.columns

Index(['Entity', 'Code', 'Year', 'Total population (Gapminder, HYDE & UN)',
       'Continent',
       'PM2.5 air pollution, mean annual exposure (micrograms per cubic meter)',
       'Deaths - Ambient particulate matter pollution - Sex: Both - Age: Age-standardized (Rate)'],
      dtype='object')

**Nhận xét:** Nhiều thuộc tính có tên dài khiến việc truy xuất dữ liệu vào các thuộc tính đó dễ sai nên sẽ lấy tên thuộc tính ngắn lại.

In [13]:
def handle_name_columns(name_columns):
  list_name =[]
  for column in name_columns.tolist():
    index = column.find(' (')
    if index !=-1:
      list_name.append(column[:index])
    else:
      list_name.append(column)
  return list_name

In [14]:
list_name_columns = handle_name_columns(death_pm_df.columns)
death_pm_df = death_pm_df.rename(columns =dict(zip(death_pm_df.columns,list_name_columns)))

In [15]:
#sau khi đổi tên cột
death_pm_df.columns

Index(['Entity', 'Code', 'Year', 'Total population', 'Continent',
       'PM2.5 air pollution, mean annual exposure',
       'Deaths - Ambient particulate matter pollution - Sex: Both - Age: Age-standardized'],
      dtype='object')

## Dữ liệu có bao nhiêu dòng và bao nhiêu cột?

In [16]:
num_rows, num_cols = death_pm_df.shape

In [17]:
(num_rows, num_cols)

(8708, 7)

## Mỗi dòng có ý nghĩa gì? Có vấn đề các dòng có ý nghĩa khác nhau không?

### Dữ liệu có các dòng bị lặp không?

In [18]:
death_pm_df.duplicated().sum()

0

## Xem giá trị thiếu mỗi cột

In [19]:
death_pm_df.isna().sum()

Entity                                                                                  0
Code                                                                                 1607
Year                                                                                    0
Total population                                                                     1448
Continent                                                                            8423
PM2.5 air pollution, mean annual exposure                                            6068
Deaths - Ambient particulate matter pollution - Sex: Both - Age: Age-standardized    2240
dtype: int64

### Xử lý giá trị thiếu thuộc tính continent

In [20]:
code_continent_df = death_pm_df[['Entity','Code','Continent']]
code_continent_df = code_continent_df.dropna(axis=0, how='any').reset_index(drop=True)
code_continent_df

,Entity,Code,Continent
0,Abkhazia,OWID_ABK,Asia
1,Afghanistan,AFG,Asia
2,Akrotiri and Dhekelia,OWID_AKD,Asia
3,Albania,ALB,Europe
4,Algeria,DZA,Africa
...,...,...,...
280,Yugoslavia,OWID_YGS,Europe
281,Zambia,ZMB,Africa
282,Zanzibar,OWID_ZAN,Africa
283,Zimbabwe,ZWE,Africa


In [21]:
death_pm_df = death_pm_df.merge(code_continent_df,how='left',left_on=['Entity','Code'],right_on=['Entity','Code'])
death_pm_df = death_pm_df.drop(columns='Continent_x',axis=1)
death_pm_df.rename(columns={'Continent_y':'Continent'},inplace = True)
death_pm_df.head()

,Entity,Code,Year,Total population,"PM2.5 air pollution, mean annual exposure",Deaths - Ambient particulate matter pollution - Sex: Both - Age: Age-standardized,Continent
0,Abkhazia,OWID_ABK,2015.0,NaN,NaN,NaN,Asia
1,Afghanistan,AFG,1990.0,12412000.0,60.93669,46.446589,Asia
2,Afghanistan,AFG,1991.0,13299000.0,NaN,46.033841,Asia
3,Afghanistan,AFG,1992.0,14486000.0,NaN,44.243766,Asia
4,Afghanistan,AFG,1993.0,15817000.0,NaN,44.440148,Asia


In [22]:
# Kiểm tra giá trị thiếu
death_pm_df.isna().sum()

Entity                                                                                  0
Code                                                                                 1607
Year                                                                                    0
Total population                                                                     1448
PM2.5 air pollution, mean annual exposure                                            6068
Deaths - Ambient particulate matter pollution - Sex: Both - Age: Age-standardized    2240
Continent                                                                            1637
dtype: int64

In [23]:
# Sau khi điền giá trị nan cho cột continent thì vẫn còn các dòng bị nan
continent_nan_df = death_pm_df[(death_pm_df['Continent'].isna()) | (death_pm_df['Code'].isna())]
continent_nan_df.head()

,Entity,Code,Year,Total population,"PM2.5 air pollution, mean annual exposure",Deaths - Ambient particulate matter pollution - Sex: Both - Age: Age-standardized,Continent
31,Africa,NaN,1990.0,630350016.0,NaN,NaN,NaN
32,Africa,NaN,1991.0,647344000.0,NaN,NaN,NaN
33,Africa,NaN,1992.0,664512000.0,NaN,NaN,NaN
34,Africa,NaN,1993.0,681868992.0,NaN,NaN,NaN
35,Africa,NaN,1994.0,699444992.0,NaN,NaN,NaN


**Nhận xét:** Nhận thấy họ tính trên toàn bộ châu lục đó và thế giới nên continent bị nan đều dễ hiểu. Ta sẽ xóa bỏ những dòng này 

In [24]:
# xóa các dòng thuộc tính continent bị nan
death_pm_df = death_pm_df[(~death_pm_df['Continent'].isna()) & (~death_pm_df['Code'].isna())].reset_index(drop=True)

In [25]:
death_pm_df.shape

(7071, 7)

In [26]:
death_pm_df.isna().sum()

Entity                                                                                  0
Code                                                                                    0
Year                                                                                    0
Total population                                                                       51
PM2.5 air pollution, mean annual exposure                                            4948
Deaths - Ambient particulate matter pollution - Sex: Both - Age: Age-standardized    1611
Continent                                                                               0
dtype: int64

**Nhận xét:** Các thuộc tính entity, code, continent không còn giá trị thiếu nữa.

## Xử lý dữ liệu gdp_death_rate

In [27]:
gdp_death_rate_df.head()

,Entity,Code,Year,"Total population (Gapminder, HYDE & UN)",Continent,"GDP per capita, PPP (constant 2011 international $)",Death rate – Outdoor air pollution (age-standardized) (IHME)
0,Abkhazia,OWID_ABK,2015,NaN,Asia,NaN,NaN
1,Afghanistan,AFG,1800,3280000.0,NaN,NaN,NaN
2,Afghanistan,AFG,1801,3280000.0,NaN,NaN,NaN
3,Afghanistan,AFG,1802,3280000.0,NaN,NaN,NaN
4,Afghanistan,AFG,1803,3280000.0,NaN,NaN,NaN


In [28]:
#Lấy dữ liệu từ năm 1990 đến nay
gdp_death_rate_df['Year'] =gdp_death_rate_df['Year'].apply(get_year)
gdp_death_rate_df =gdp_death_rate_df[~gdp_death_rate_df['Year'].isna()].reset_index(drop=True)
gdp_death_rate_df.head()

,Entity,Code,Year,"Total population (Gapminder, HYDE & UN)",Continent,"GDP per capita, PPP (constant 2011 international $)",Death rate – Outdoor air pollution (age-standardized) (IHME)
0,Abkhazia,OWID_ABK,2015.0,NaN,Asia,NaN,NaN
1,Afghanistan,AFG,1990.0,12412000.0,NaN,NaN,52.06
2,Afghanistan,AFG,1991.0,13299000.0,NaN,NaN,51.64
3,Afghanistan,AFG,1992.0,14486000.0,NaN,NaN,49.86
4,Afghanistan,AFG,1993.0,15817000.0,NaN,NaN,50.10


In [29]:
# Đổi tên thuộc tính
list_name_columns = handle_name_columns(gdp_death_rate_df.columns)
gdp_death_rate_df = gdp_death_rate_df.rename(columns =dict(zip(gdp_death_rate_df.columns,list_name_columns)))

In [30]:
# Điền giá trị thiếu cho continent với code
gdp_death_rate_df = gdp_death_rate_df.merge(code_continent_df,how='left',left_on=['Entity','Code'],right_on=['Entity','Code'])
gdp_death_rate_df = gdp_death_rate_df.drop(columns='Continent_x',axis=1)
gdp_death_rate_df.rename(columns={'Continent_y':'Continent'},inplace = True)
gdp_death_rate_df.head()

,Entity,Code,Year,Total population,"GDP per capita, PPP",Death rate – Outdoor air pollution,Continent
0,Abkhazia,OWID_ABK,2015.0,NaN,NaN,NaN,Asia
1,Afghanistan,AFG,1990.0,12412000.0,NaN,52.06,Asia
2,Afghanistan,AFG,1991.0,13299000.0,NaN,51.64,Asia
3,Afghanistan,AFG,1992.0,14486000.0,NaN,49.86,Asia
4,Afghanistan,AFG,1993.0,15817000.0,NaN,50.10,Asia


In [31]:
# xóa các dòng thuộc tính continent bị nan
gdp_death_rate_df = gdp_death_rate_df[(~gdp_death_rate_df['Continent'].isna())&(~gdp_death_rate_df['Code'].isna())].reset_index(drop=True)

In [32]:
gdp_death_rate_df.shape

(7088, 7)

In [33]:
#Kiểm tra lại giá trị thiếu
gdp_death_rate_df.isna().sum()

Entity                                   0
Code                                     0
Year                                     0
Total population                        68
GDP per capita, PPP                   1949
Death rate – Outdoor air pollution    1628
Continent                                0
dtype: int64

# Gộp data

In [34]:
data_df = death_pm_df.merge(gdp_death_rate_df,how='left',left_on=['Entity','Code','Year','Continent','Total population'],
                            right_on=['Entity','Code','Year','Continent','Total population'])

In [35]:
data_df

,Entity,Code,Year,Total population,"PM2.5 air pollution, mean annual exposure",Deaths - Ambient particulate matter pollution - Sex: Both - Age: Age-standardized,Continent,"GDP per capita, PPP",Death rate – Outdoor air pollution
0,Abkhazia,OWID_ABK,2015.0,NaN,NaN,NaN,Asia,NaN,NaN
1,Afghanistan,AFG,1990.0,12412000.0,60.936690,46.446589,Asia,NaN,52.06
2,Afghanistan,AFG,1991.0,13299000.0,NaN,46.033841,Asia,NaN,51.64
3,Afghanistan,AFG,1992.0,14486000.0,NaN,44.243766,Asia,NaN,49.86
4,Afghanistan,AFG,1993.0,15817000.0,NaN,44.440148,Asia,NaN,50.10
...,...,...,...,...,...,...,...,...,...
7066,Zimbabwe,ZWE,2016.0,14030000.0,24.513035,25.643570,Africa,1879.628119,29.76
7067,Zimbabwe,ZWE,2017.0,14237000.0,NaN,26.166182,Africa,1899.774977,30.22
7068,Zimbabwe,ZWE,2018.0,14439000.0,NaN,NaN,Africa,NaN,NaN
7069,Zimbabwe,ZWE,2019.0,14645000.0,NaN,NaN,Africa,NaN,NaN


#Xét tập dữ liệu GDP_Population_Rate

In [36]:
gdp_population_rate_df = pd.read_csv('GDP_Population_Rate.csv')

In [37]:
gdp_population_rate_df = gdp_population_rate_df[~gdp_population_rate_df['Code'].isna()].reset_index(drop=True)
gdp_population_rate_df.head()

,Entity,Code,Year,"Air pollution (total) (IHME, 2019)"
0,Afghanistan,AFG,1990,13.56
1,Afghanistan,AFG,1991,13.19
2,Afghanistan,AFG,1992,13.05
3,Afghanistan,AFG,1993,12.88
4,Afghanistan,AFG,1994,12.80


In [38]:
gdp_population_rate_df['Year'].unique()

array([1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000,
       2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011,
       2012, 2013, 2014, 2015, 2016, 2017])

In [39]:
gdp_population_rate_df.shape

(5432, 4)

#Xét tập dữ liệu CO2_Emission

In [40]:
c02_emission_df = pd.read_csv('CO2_emission.csv')
c02_emission_df.head()

,Entity,Code,Year,Annual CO2 emissions
0,Afghanistan,AFG,1949,0.014656
1,Afghanistan,AFG,1950,0.084272
2,Afghanistan,AFG,1951,0.091600
3,Afghanistan,AFG,1952,0.091600
4,Afghanistan,AFG,1953,0.106256


In [41]:
# Lấy dữ liệu từ năm 1990
c02_emission_df['Year'] = c02_emission_df['Year'].apply(get_year)
c02_emission_df = c02_emission_df[~c02_emission_df['Year'].isna()].reset_index(drop=True)
c02_emission_df.head()

,Entity,Code,Year,Annual CO2 emissions
0,Afghanistan,AFG,1990.0,2.602408
1,Afghanistan,AFG,1991.0,2.426486
2,Afghanistan,AFG,1992.0,1.381522
3,Afghanistan,AFG,1993.0,1.333837
4,Afghanistan,AFG,1994.0,1.282457


In [47]:
c02_emission_df.shape

(6950, 4)

# Xét tập dữ liệu Enviroment Rish

In [42]:
enviroment_risk_df = pd.read_csv('Enviroment_Risk.csv')

In [43]:
enviroment_risk_df

,"<!DOCTYPE html><html><head><title>Google Drive - Virus scan warning</title><meta http-equiv=""content-type"" content=""text/html; charset=utf-8""/><link href=&#47;static&#47;doclist&#47;client&#47;css&#47;2149812255&#45;untrustedcontent.css rel=""stylesheet"" nonce=""wXF+0dIS9Vil2LDvloGxzw""><link rel=""icon"" href=""//ssl.gstatic.com/images/branding/product/1x/drive_2020q4_32dp.png""/><style nonce=""wXF+0dIS9Vil2LDvloGxzw"">#gbar",#guser{font-size:13px;padding-top:0px !important;}#gbar{height:22px}#guser{padding-bottom:7px !important;text-align:right}.gbh,.gbd{border-top:1px solid #c9d7f1;font-size:1px}.gbh{height:0;position:absolute;top:24px;width:100%}@media all{.gb1{height:22px;margin-right:.5em;vertical-align:top}#gbar{float:left}}a.gb1,a.gb4{text-decoration:underline !important}a.gb1,a.gb4{color:#00c !important}.gbi .gb4{color:#dd8e27 !important}.gbf .gb4{color:#900 !important}
0,"</style><script nonce=""t+74qmOf63gaZJnrk2DZhg""...",NaN,NaN,NaN,NaN


In [44]:
data_df[data_df['Continent'] == 'Asia']['Entity'].unique()

array(['Abkhazia', 'Afghanistan', 'Akrotiri and Dhekelia', 'Armenia',
       'Azerbaijan', 'Bahrain', 'Bangladesh', 'Bhutan',
       'British Indian Ocean Territory', 'Brunei', 'Cambodia', 'China',
       'Christmas Island', 'Cocos Islands', 'Georgia', 'Hong Kong',
       'India', 'Indonesia', 'Iran', 'Iraq', 'Israel', 'Japan', 'Jordan',
       'Kazakhstan', 'Kuwait', 'Kyrgyzstan', 'Laos', 'Lebanon', 'Macao',
       'Malaysia', 'Maldives', 'Mongolia', 'Myanmar', 'Nagorno-Karabakh',
       'Nepal', 'North Korea', 'Northern Cyprus', 'Oman', 'Pakistan',
       'Palestine', 'Philippines', 'Qatar', 'Republic of Vietnam',
       'Saudi Arabia', 'Singapore', 'South Korea', 'South Ossetia',
       'Sri Lanka', 'Syria', 'Taiwan', 'Tajikistan', 'Thailand', 'Timor',
       'Turkey', 'Turkmenistan', 'United Arab Emirates', 'United Korea',
       'Uzbekistan', 'Vietnam', 'Yemen', 'Yemen Arab Republic',
       "Yemen People's Republic"], dtype=object)

In [45]:
data_df['Code'].nunique()

285

In [46]:
c02_emission_df['Code'].nunique()

223